In [1]:
import pandas as pd
import datetime as dt
import yfinance as yf

In [2]:
# Define the time period and stock symbol
start = dt.datetime(2021, 6, 1)
end = dt.datetime(2022, 1, 5)
symbol = 'ZOMATO.NS'

# Download stock data
stk_data = yf.download(symbol, start=start, end=end)

[*********************100%***********************]  1 of 1 completed


In [3]:
# Selecting specific columns
stk_data = stk_data[["Open", "High", "Low", "Close"]]

In [4]:
stk_data 

Price,Open,High,Low,Close
Ticker,ZOMATO.NS,ZOMATO.NS,ZOMATO.NS,ZOMATO.NS
Date,,,,
2021-07-23,116.000000,138.899994,115.000000,126.000000
2021-07-26,126.349998,143.750000,125.300003,140.649994
2021-07-27,141.699997,147.800003,127.750000,132.899994
2021-07-28,131.000000,135.000000,123.550003,131.199997
2021-07-29,134.949997,144.000000,132.199997,141.550003
...,...,...,...,...
2021-12-29,132.699997,137.699997,131.550003,137.100006
2021-12-30,137.050003,137.050003,133.300003,133.949997


In [5]:
# Adjust to your column names
new_headers = ['Open','High','Low','Close'] 
stk_data.columns = new_headers

In [6]:
stk_data

,Open,High,Low,Close
Date,,,,
2021-07-23,116.000000,138.899994,115.000000,126.000000
2021-07-26,126.349998,143.750000,125.300003,140.649994
2021-07-27,141.699997,147.800003,127.750000,132.899994
2021-07-28,131.000000,135.000000,123.550003,131.199997
2021-07-29,134.949997,144.000000,132.199997,141.550003
...,...,...,...,...
2021-12-29,132.699997,137.699997,131.550003,137.100006
2021-12-30,137.050003,137.050003,133.300003,133.949997
2021-12-31,133.850006,138.399994,133.399994,137.399994


In [7]:
stk_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 113 entries, 2021-07-23 to 2022-01-04
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    113 non-null    float64
 1   High    113 non-null    float64
 2   Low     113 non-null    float64
 3   Close   113 non-null    float64
dtypes: float64(4)
memory usage: 4.4 KB


In [8]:
#column="Close"

In [9]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

Len: (113, 4)


In [10]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])

In [11]:
data1

,Open,High,Low,Close
0,0.000000,0.280764,0.000000,0.048544
1,0.229236,0.396655,0.262420,0.454924
2,0.569214,0.493429,0.324841,0.239944
3,0.332226,0.187575,0.217834,0.192788
4,0.419712,0.402628,0.438216,0.479889
...,...,...,...,...
108,0.369878,0.252091,0.421656,0.356450
109,0.466224,0.236559,0.466242,0.269071
110,0.395349,0.268817,0.468790,0.364771
111,0.524917,0.365591,0.593631,0.474341


In [12]:
import warnings
warnings.filterwarnings("ignore")

In [13]:
training_size = round(len(data1 ) * 0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

90
X_train length: (90, 4)
X_test length: (23, 4)
y_train length: (90, 4)
y_test length: (23, 4)


In [14]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [15]:
performance

{'Model': [], 'RMSE': [], 'MaPe': [], 'Lag': [], 'Test': []}

In [16]:
def cominbation(dataset1,listt):
    print(listt)
    datasetTwo=dataset1[listt]
    test_obs = 28
    train =datasetTwo[:-test_obs]
    test = datasetTwo[-test_obs:]
    from statsmodels.tsa.api import VAR
    for i in [1,2,3,4,5,6,7,8,9,10]:
        model = VAR(train)
        results = model.fit(i)
        print('Order =', i)
        print('AIC: ', results.aic) #Akaike Information Criterion (AIC)
        print('BIC: ', results.bic) # Bayesian Information Criterion
        print()
    x = model.select_order(maxlags=12)
    order=x.selected_orders["aic"]
    result = model.fit(order)
    #result.summary()
    lagged_Values = train.values[-order:]
    pred = result.forecast(y=lagged_Values,steps=28) 
    preds=pd.DataFrame(pred,columns=listt)
    preds.to_csv("varforecasted_{}.csv".format(test_obs))
    from sklearn.metrics import mean_squared_error
    rmse= round(mean_squared_error(test,pred,squared=False))
    from sklearn.metrics import mean_absolute_percentage_error
    mape=mean_absolute_percentage_error(test,pred)
    performance["Model"].append(listt)
    performance["RMSE"].append(rmse)
    performance["MaPe"].append(mape)
    performance["Lag"].append(order)
    performance["Test"].append(test_obs)
    perf=pd.DataFrame(performance)  
    return perf,result,pred

In [24]:
listt=["Close","High","Open","Low"]
#listt=["AQI_calculated","PM10","PM2.5","NOx","NO2","NO","NH3","SO2","CO",'year']

In [23]:
perf,result,pred=cominbation(data1,listt)

['AQI_calculated', 'PM10', 'PM2.5', 'NOx', 'NO2', 'NO', 'NH3', 'SO2', 'CO', 'year']


KeyError: "None of [Index(['AQI_calculated', 'PM10', 'PM2.5', 'NOx', 'NO2', 'NO', 'NH3', 'SO2',\n       'CO', 'year'],\n      dtype='object')] are in the [columns]"

In [19]:
perf,result,pred=cominbation(data1,listt)

['Close', 'High', 'Open', 'Low']
Order = 1
AIC:  -22.78529898606033
BIC:  -22.206533081573824

Order = 2
AIC:  -22.78790424731839
BIC:  -21.738768562009025

Order = 3
AIC:  -22.815593741821917
BIC:  -21.28938153623971

Order = 4
AIC:  -22.75797259008421
BIC:  -20.74781774418636

Order = 5
AIC:  -22.67677519582047
BIC:  -20.175647229412895

Order = 6
AIC:  -22.588223104190067
BIC:  -19.58892202511789

Order = 7
AIC:  -22.54096735094641
BIC:  -19.036118326638814

Order = 8
AIC:  -22.607464505713054
BIC:  -18.589512353963883

Order = 9
AIC:  -22.546933207904036
BIC:  -18.008136703135918

Order = 10
AIC:  -23.02370780639564
BIC:  -17.956133798129578



In [20]:
data1

,Open,High,Low,Close
0,0.000000,0.280764,0.000000,0.048544
1,0.229236,0.396655,0.262420,0.454924
2,0.569214,0.493429,0.324841,0.239944
3,0.332226,0.187575,0.217834,0.192788
4,0.419712,0.402628,0.438216,0.479889
...,...,...,...,...
108,0.369878,0.252091,0.421656,0.356450
109,0.466224,0.236559,0.466242,0.269071
110,0.395349,0.268817,0.468790,0.364771
111,0.524917,0.365591,0.593631,0.474341


In [21]:
perf

,Model,RMSE,MaPe,Lag,Test
0,"[Close, High, Open, Low]",0,0.355106,1,28
1,"[Close, High, Open, Low]",0,0.355106,1,28
